In [1]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om

import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, identity
from scipy.sparse.linalg import spsolve

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline

In [4]:
def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh)


In [32]:
mesh = read_mesh_om('cube.obj')

In [26]:
write_mesh_om(mesh, 'test/cube-copy.obj')

In [24]:
mesh.has_vertex_colors()

True

In [33]:
print(mesh.vertices())
set = False
for v_it in mesh.vertices():
    if not set:
        mesh.set_point(v_it, np.array([-1, -1 , 1]))
        set = True
    print(mesh.point(v_it), type(mesh.point(v_it)))
    mesh.set_color(v_it, np.array([255.,182.,193.,255.])/255.)

for f_it in mesh.faces():
    mesh.set_color(f_it, np.array([255.,182.,193.,255.])/255.)
for c_it in mesh.face_colors():
    print(c_it, type(c_it))
write_mesh_om(mesh, 'test/cube-copy.obj')

[-1. -1.  1.] <class 'numpy.ndarray'>
[ 0.5 -0.5  0.5] <class 'numpy.ndarray'>
[-0.5  0.5  0.5] <class 'numpy.ndarray'>
[0.5 0.5 0.5] <class 'numpy.ndarray'>
[-0.5  0.5 -0.5] <class 'numpy.ndarray'>
[ 0.5  0.5 -0.5] <class 'numpy.ndarray'>
[-0.5 -0.5 -0.5] <class 'numpy.ndarray'>
[ 0.5 -0.5 -0.5] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <class 'numpy.ndarray'>
[1.         0.7137255  0.75686276 1.        ] <c